In [1]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn import model_selection
from sklearn.datasets import load_iris
from sklearn.datasets import make_blobs
from matplotlib.colors import ListedColormap

# Урок 8. Снижение размерности данных.

**1. Обучить любую модель классификации на датасете IRIS до применения PCA и после него. Сравнить качество классификации по отложенной выборке.**

**2*. Написать свою реализацию метода главных компонент с помощью сингулярного разложения с использованием функции numpy.linalg.svd().**


С увеличением числа кластеров, среднеквадратичное внутрикластерное расстояние уменьшается. Похоже что нельзя рассматривать эту метрику как критерий качества модели без учёта межкластерных расстояний.